# Getting Data from Twitter 

This notebook was done with the help of [this tutorial](https://towardsdatascience.com/how-to-access-data-from-the-twitter-api-using-tweepy-python-e2d9e4d54978).

## Loading Required Keys and Tokens from Twitter Website
The researcher got keys and tokens from Twitter and put it in a file named `.env` to make sure that he can publish the code while using it for the project.

In [1]:
from dotenv import dotenv_values

# Loads values from the .env file
config = dotenv_values(".env")

## Importing Tweepy
[Tweepy](https://docs.tweepy.org/en/stable/getting_started.html#introduction) is a common library used to communicate with Twitter using Python.

In [2]:
import tweepy

## Initiate Tweepy Objects
Tweepy objects are made for getting data in an organized manner when using Python.

In [3]:
client = tweepy.Client(bearer_token = config["bearer_token"])
auth = tweepy.OAuth1UserHandler(
    config["api_key"],
    config["api_key_secret"],
    config["access_token"],
    config["access_token_secret"]
)
api = tweepy.API(auth)

## Getting Information about a User
This part of the notebook demonstrates how the researcher got information about a user. More information about using Tweepy to get user information can be found [here](https://docs.tweepy.org/en/stable/api.html#tweepy.API.get_user).

In [4]:
# Get a user by their user id
test_user = api.get_user(user_id = 787405734442958848)
test_user._json

{'id': 787405734442958848,
 'id_str': '787405734442958848',
 'name': 'The Best In Dumbest',
 'screen_name': 'best_in_dumbest',
 'location': '',
 'profile_location': None,
 'description': 'Blame @MoistHorse, Inspired by @MakingInvisible, using cmu phonetic data to produce incongruous matches. Some images via Lorem Flickr.',
 'url': None,
 'entities': {'description': {'urls': []}},
 'protected': False,
 'followers_count': 1995,
 'friends_count': 5,
 'listed_count': 58,
 'created_at': 'Sat Oct 15 21:32:11 +0000 2016',
 'favourites_count': 5,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': False,
 'verified': False,
 'statuses_count': 17209,
 'lang': None,
 'status': {'created_at': 'Sun Oct 23 00:00:03 +0000 2022',
  'id': 1583971477434818561,
  'id_str': '1583971477434818561',
  'text': 'Be the "Soot"\nin "Soothsayers" https://t.co/BAFX4zJuOc',
  'truncated': False,
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [],
   'urls': [],
   'media': [{'id': 1583971

## Gathering Information about Twitter Profiles from Kaggle
In a paper entitled [Enhanced Twitter bot detection using ensemble machine learning](https://ieeexplore.ieee.org/document/9358734), the researchers got the IDs of Twitter accounts from a [Kaggle repository](https://www.kaggle.com/datasets/davidmartngutirrez/twitter-bots-accounts). After getting them, they derived the data shown in [this image](https://ieeexplore.ieee.org/mediastore_new/IEEE/content/media/9358464/9358467/9358734/9358734-table-1-source-large.gif). The researcher of this project will do something similar.

### Importing Data about Twitter IDs

In [5]:
# Import library for processing tabular data
import pandas as pd

# Import dataframe of Twitter IDs and if the Twitter IDs refer to bots
twitter_df = pd.read_csv("twitter_human_bots_dataset.csv")

# Preview dataframe
twitter_df.head()

,id,account_type
0,787405734442958848,bot
1,796216118331310080,human
2,875949740503859204,human
3,756119643622735875,human
4,464781334,human


### Note Data about Twitter Accounts
The researcher made empty columns before reading the data.

In [6]:
twitter_df["created_at"] = None
twitter_df["default_profile"] = None
twitter_df["default_profile_image"] = None
twitter_df["description"] = None
twitter_df["favourites_count"] = None
twitter_df["followers_count"] = None
twitter_df["geo_enabled"] = None
twitter_df["lang"] = None
twitter_df["location"] = None
twitter_df["profile_background_image"] = None
twitter_df["profile_image"] = None
twitter_df["screen_name"] = None
twitter_df["statuses_count"] = None
twitter_df["verified"] = None
twitter_df["average_tweets_per_day"] = None # Computed based on data and not only from API response
twitter_df["account_age"] = None

In [7]:
twitter_df.head()

,id,account_type,created_at,default_profile,default_profile_image,description,favourites_count,followers_count,geo_enabled,lang,location,profile_background_image,profile_image,screen_name,statuses_count,verified,average_tweets_per_day,account_age
0,787405734442958848,bot,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,796216118331310080,human,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,875949740503859204,human,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,756119643622735875,human,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,464781334,human,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


Once the researcher made the columns, he also assigned values.

In [10]:
import time

twit_ids = twitter_df["id"].to_list()
for i in range(2500, len(twit_ids)):
    time.sleep(3.5) # Make sure that researcher does not exceed API rate limit
    print("{} / {}".format(i + 1, len(twit_ids)))
    try:
        twit_user = api.get_user(user_id = twit_ids[i])
        twit_data = twit_user._json
        
        twitter_df.iloc[i,2] = twit_data["created_at"]
        twitter_df.iloc[i,3] = twit_data["default_profile"]
        twitter_df.iloc[i,4] = twit_data["default_profile_image"]
        twitter_df.iloc[i,5] = twit_data["description"]
        twitter_df.iloc[i,6] = twit_data["favourites_count"]
        twitter_df.iloc[i,7] = twit_data["followers_count"]
        twitter_df.iloc[i,8] = twit_data["geo_enabled"]
        twitter_df.iloc[i,9] = twit_data["lang"]
        twitter_df.iloc[i,10] = twit_data["location"]
        twitter_df.iloc[i,11] = twit_data["profile_background_image_url"]
        twitter_df.iloc[i,12] = twit_data["profile_image_url"]
        twitter_df.iloc[i,13] = twit_data["screen_name"]
        twitter_df.iloc[i,14] = twit_data["statuses_count"]
        twitter_df.iloc[i,15] = twit_data["verified"]
        # twitter_df.iloc[i,16] = twit_data["average_tweets_per_day"] # Skipped for now
        # twitter_df.iloc[i,17] = twit_data["average_tweets_per_day"] # Skipped for now
        
        # Save data every 1000 lines of code
        if i % 1000 == 1: twitter_df.to_csv("twitter_output.csv", index=False)
    except Exception as e:
        print(e)
        print("Can't get user with ID of {}".format(twit_ids[i]))

2501 / 37438
2502 / 37438
2503 / 37438
2504 / 37438
2505 / 37438
2506 / 37438
2507 / 37438
2508 / 37438
2509 / 37438
2510 / 37438
2511 / 37438
2512 / 37438
2513 / 37438
2514 / 37438
2515 / 37438
2516 / 37438
2517 / 37438
2518 / 37438
2519 / 37438
2520 / 37438
2521 / 37438
2522 / 37438
2523 / 37438
2524 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 4170204321
2525 / 37438
2526 / 37438
2527 / 37438
2528 / 37438
2529 / 37438
2530 / 37438
2531 / 37438
2532 / 37438
2533 / 37438
2534 / 37438
2535 / 37438
2536 / 37438
2537 / 37438
2538 / 37438
2539 / 37438
2540 / 37438
2541 / 37438
2542 / 37438
2543 / 37438
2544 / 37438
2545 / 37438
2546 / 37438
2547 / 37438
2548 / 37438
2549 / 37438
2550 / 37438
2551 / 37438
2552 / 37438
2553 / 37438
2554 / 37438
2555 / 37438
2556 / 37438
2557 / 37438
2558 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 717812608942866432
2559 / 37438
2560 / 37438
2561 / 37438
2562 / 37438
2563 / 37438
2564 / 37438
2565 / 37438
256

403 Forbidden
63 - User has been suspended.
Can't get user with ID of 1378111975
2996 / 37438
2997 / 37438
2998 / 37438
2999 / 37438
3000 / 37438
3001 / 37438
3002 / 37438
[Errno 13] Permission denied: 'twitter_output.csv'
Can't get user with ID of 1055406669356826626
3003 / 37438
3004 / 37438
3005 / 37438
3006 / 37438
3007 / 37438
3008 / 37438
3009 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 1554010706
3010 / 37438
3011 / 37438
3012 / 37438
3013 / 37438
3014 / 37438
3015 / 37438
3016 / 37438
3017 / 37438
3018 / 37438
3019 / 37438
3020 / 37438
3021 / 37438
3022 / 37438
3023 / 37438
3024 / 37438
3025 / 37438
3026 / 37438
3027 / 37438
3028 / 37438
3029 / 37438
3030 / 37438
3031 / 37438
3032 / 37438
3033 / 37438
3034 / 37438
3035 / 37438
3036 / 37438
3037 / 37438
3038 / 37438
3039 / 37438
3040 / 37438
3041 / 37438
3042 / 37438
3043 / 37438
3044 / 37438
3045 / 37438
3046 / 37438
3047 / 37438
3048 / 37438
3049 / 37438
3050 / 37438
3051 / 37438
3052 / 37438
3053 / 37

3472 / 37438
403 Forbidden
63 - User has been suspended.
Can't get user with ID of 206593863
3473 / 37438
3474 / 37438
3475 / 37438
3476 / 37438
3477 / 37438
3478 / 37438
3479 / 37438
3480 / 37438
3481 / 37438
3482 / 37438
3483 / 37438
3484 / 37438
3485 / 37438
3486 / 37438
3487 / 37438
3488 / 37438
3489 / 37438
3490 / 37438
3491 / 37438
3492 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 196108224
3493 / 37438
3494 / 37438
403 Forbidden
63 - User has been suspended.
Can't get user with ID of 22044633
3495 / 37438
3496 / 37438
3497 / 37438
3498 / 37438
3499 / 37438
3500 / 37438
3501 / 37438
3502 / 37438
3503 / 37438
3504 / 37438
3505 / 37438
3506 / 37438
3507 / 37438
3508 / 37438
3509 / 37438
3510 / 37438
3511 / 37438
3512 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 838908851973984256
3513 / 37438
3514 / 37438
3515 / 37438
3516 / 37438
3517 / 37438
3518 / 37438
403 Forbidden
63 - User has been suspended.
Can't get user with ID of 41705570


3945 / 37438
3946 / 37438
3947 / 37438
3948 / 37438
3949 / 37438
3950 / 37438
3951 / 37438
3952 / 37438
3953 / 37438
3954 / 37438
3955 / 37438
3956 / 37438
3957 / 37438
3958 / 37438
3959 / 37438
3960 / 37438
3961 / 37438
3962 / 37438
3963 / 37438
3964 / 37438
3965 / 37438
3966 / 37438
3967 / 37438
3968 / 37438
3969 / 37438
3970 / 37438
3971 / 37438
3972 / 37438
3973 / 37438
3974 / 37438
3975 / 37438
3976 / 37438
3977 / 37438
3978 / 37438
3979 / 37438
3980 / 37438
3981 / 37438
3982 / 37438
3983 / 37438
3984 / 37438
3985 / 37438
3986 / 37438
3987 / 37438
3988 / 37438
3989 / 37438
3990 / 37438
3991 / 37438
3992 / 37438
3993 / 37438
3994 / 37438
3995 / 37438
3996 / 37438
3997 / 37438
3998 / 37438
403 Forbidden
63 - User has been suspended.
Can't get user with ID of 3902565862
3999 / 37438
4000 / 37438
4001 / 37438
4002 / 37438
4003 / 37438
4004 / 37438
4005 / 37438
4006 / 37438
4007 / 37438
4008 / 37438
4009 / 37438
4010 / 37438
4011 / 37438
4012 / 37438
404 Not Found
50 - User not found.


4414 / 37438
4415 / 37438
4416 / 37438
4417 / 37438
4418 / 37438
4419 / 37438
4420 / 37438
4421 / 37438
4422 / 37438
4423 / 37438
4424 / 37438
4425 / 37438
4426 / 37438
4427 / 37438
4428 / 37438
4429 / 37438
4430 / 37438
4431 / 37438
4432 / 37438
4433 / 37438
4434 / 37438
4435 / 37438
4436 / 37438
4437 / 37438
4438 / 37438
4439 / 37438
4440 / 37438
4441 / 37438
4442 / 37438
4443 / 37438
4444 / 37438
4445 / 37438
4446 / 37438
4447 / 37438
4448 / 37438
4449 / 37438
4450 / 37438
4451 / 37438
4452 / 37438
4453 / 37438
4454 / 37438
4455 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 947841424229142528
4456 / 37438
4457 / 37438
4458 / 37438
4459 / 37438
4460 / 37438
4461 / 37438
4462 / 37438
4463 / 37438
4464 / 37438
4465 / 37438
4466 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 2965142314
4467 / 37438
4468 / 37438
4469 / 37438
4470 / 37438
4471 / 37438
4472 / 37438
4473 / 37438
4474 / 37438
4475 / 37438
4476 / 37438
4477 / 37438
4478 / 37438
447

...

35301 / 37438
35302 / 37438
35303 / 37438
35304 / 37438
35305 / 37438
35306 / 37438
35307 / 37438
35308 / 37438
35309 / 37438
35310 / 37438
35311 / 37438
35312 / 37438
35313 / 37438
35314 / 37438
35315 / 37438
35316 / 37438
35317 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 811293391728189440
35318 / 37438
35319 / 37438
35320 / 37438
403 Forbidden
63 - User has been suspended.
Can't get user with ID of 747069301492850689
35321 / 37438
35322 / 37438
35323 / 37438
35324 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 827902169051340802
35325 / 37438
35326 / 37438
35327 / 37438
35328 / 37438
35329 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 14140714
35330 / 37438
35331 / 37438
35332 / 37438
35333 / 37438
35334 / 37438
35335 / 37438
35336 / 37438
35337 / 37438
35338 / 37438
35339 / 37438
35340 / 37438
35341 / 37438
35342 / 37438
35343 / 37438
35344 / 37438
35345 / 37438
35346 / 37438
35347 / 37438
35348 / 37438
35349 / 3

35744 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 1030692350597652481
35745 / 37438
35746 / 37438
35747 / 37438
35748 / 37438
35749 / 37438
35750 / 37438
35751 / 37438
35752 / 37438
35753 / 37438
35754 / 37438
35755 / 37438
35756 / 37438
35757 / 37438
35758 / 37438
35759 / 37438
35760 / 37438
35761 / 37438
35762 / 37438
35763 / 37438
35764 / 37438
35765 / 37438
35766 / 37438
35767 / 37438
35768 / 37438
35769 / 37438
35770 / 37438
35771 / 37438
35772 / 37438
35773 / 37438
35774 / 37438
35775 / 37438
35776 / 37438
35777 / 37438
35778 / 37438
35779 / 37438
35780 / 37438
35781 / 37438
35782 / 37438
35783 / 37438
35784 / 37438
35785 / 37438
35786 / 37438
35787 / 37438
35788 / 37438
35789 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 291492315
35790 / 37438
35791 / 37438
35792 / 37438
35793 / 37438
35794 / 37438
35795 / 37438
35796 / 37438
35797 / 37438
35798 / 37438
35799 / 37438
35800 / 37438
35801 / 37438
35802 / 37438
35803 / 37438
35804 / 

36240 / 37438
36241 / 37438
36242 / 37438
36243 / 37438
36244 / 37438
36245 / 37438
36246 / 37438
36247 / 37438
36248 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 1860689335
36249 / 37438
36250 / 37438
36251 / 37438
36252 / 37438
36253 / 37438
36254 / 37438
36255 / 37438
36256 / 37438
36257 / 37438
36258 / 37438
36259 / 37438
36260 / 37438
36261 / 37438
36262 / 37438
36263 / 37438
36264 / 37438
36265 / 37438
36266 / 37438
36267 / 37438
36268 / 37438
36269 / 37438
36270 / 37438
36271 / 37438
36272 / 37438
36273 / 37438
36274 / 37438
36275 / 37438
36276 / 37438
36277 / 37438
36278 / 37438
36279 / 37438
36280 / 37438
36281 / 37438
36282 / 37438
36283 / 37438
36284 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 2353520335
36285 / 37438
36286 / 37438
36287 / 37438
36288 / 37438
36289 / 37438
36290 / 37438
36291 / 37438
36292 / 37438
36293 / 37438
36294 / 37438
36295 / 37438
36296 / 37438
36297 / 37438
36298 / 37438
36299 / 37438
36300 / 37438
36

36722 / 37438
36723 / 37438
36724 / 37438
36725 / 37438
36726 / 37438
36727 / 37438
36728 / 37438
36729 / 37438
36730 / 37438
36731 / 37438
36732 / 37438
36733 / 37438
36734 / 37438
36735 / 37438
36736 / 37438
36737 / 37438
36738 / 37438
36739 / 37438
36740 / 37438
36741 / 37438
36742 / 37438
36743 / 37438
36744 / 37438
36745 / 37438
36746 / 37438
36747 / 37438
36748 / 37438
36749 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 57928790
36750 / 37438
36751 / 37438
36752 / 37438
36753 / 37438
36754 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 757994847869427713
36755 / 37438
36756 / 37438
36757 / 37438
36758 / 37438
36759 / 37438
36760 / 37438
36761 / 37438
36762 / 37438
36763 / 37438
36764 / 37438
36765 / 37438
36766 / 37438
36767 / 37438
36768 / 37438
36769 / 37438
36770 / 37438
36771 / 37438
36772 / 37438
36773 / 37438
36774 / 37438
36775 / 37438
36776 / 37438
36777 / 37438
36778 / 37438
36779 / 37438
36780 / 37438
36781 / 37438
403 Forbid

37211 / 37438
37212 / 37438
37213 / 37438
37214 / 37438
37215 / 37438
37216 / 37438
37217 / 37438
37218 / 37438
37219 / 37438
37220 / 37438
37221 / 37438
37222 / 37438
37223 / 37438
37224 / 37438
37225 / 37438
37226 / 37438
37227 / 37438
37228 / 37438
37229 / 37438
37230 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 1537473175
37231 / 37438
37232 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 22805168
37233 / 37438
37234 / 37438
37235 / 37438
403 Forbidden
63 - User has been suspended.
Can't get user with ID of 22280891
37236 / 37438
37237 / 37438
37238 / 37438
37239 / 37438
37240 / 37438
37241 / 37438
37242 / 37438
37243 / 37438
37244 / 37438
37245 / 37438
37246 / 37438
37247 / 37438
37248 / 37438
37249 / 37438
37250 / 37438
37251 / 37438
37252 / 37438
37253 / 37438
37254 / 37438
37255 / 37438
37256 / 37438
37257 / 37438
37258 / 37438
37259 / 37438
404 Not Found
50 - User not found.
Can't get user with ID of 865364568528363520
37260 / 37438

In [11]:
twitter_df.to_csv("twitter_output.csv", index=False)